In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) SDK: Custom tabular classification model for batch prediction with explanations

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/custom/ucaip_customjob_tabular_batchexplain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/custom/ucaip_customjob_tabular_batchexplain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Overview


This tutorial demonstrates how to use the AI Platform (Unified) Python SDK to train and deploy a custom tabular classification model for batch prediction with explanations.

### Dataset

The dataset used for this tutorial is the [Boston Housing Prices dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). The version of the dataset you will use in this tutorial is built into Tensorflow. The trained model predicts the median price of a house in units of 1K USD.

### Objective

In this notebook, you will learn how to create a custom model from a Python script in a docker container using the AI Platform (Unified) SDK, and then do a batch prediction with explanations on the uploaded model. You can alternatively create custom models from the command line using `gcloud` or online using Google Cloud Console.

The steps performed include: 

- Create an AI Platform (Unified) custom job for training a model.
- Train the model.
- View the model evaluation.
- Create a batch prediction service for the model.
- Make a batch prediction with explainability.

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the latest (preview) version of AI Platform (Unified) SDK.

In [ ]:
! pip3 install google-cloud-aiplatform

Error processing line 1 of /home/jupyter/.local/lib/python3.7/site-packages/google-cloud-aiplatform-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/conda/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of /opt/conda/lib/python3.7/site-packages/google-cloud-aiplatform-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/conda/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


Install the `cloud-storage` library as well.

In [ ]:
! pip3 install google-cloud-storage

Error processing line 1 of /home/jupyter/.local/lib/python3.7/site-packages/google-cloud-aiplatform-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/conda/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of /opt/conda/lib/python3.7/site-packages/google-cloud-aiplatform-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/conda/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


### Restart the Kernel

Once you've installed the AI Platform (Unified) SDK, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os
if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select Runtime --> Change runtime type**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs, Compute Engine APIs and Container Registry API.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,containerregistry.googleapis.com)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AI Platform Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Project ID

**If you don't know your project ID**, you might be able to get your project ID using `gcloud` command, by executing the second cell below.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: andy-1234-221921


In [ ]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AI Platform (Unified). We recommend when possible, to choose the region closest to you. 

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You can not use a Multi-Regional Storage bucket for training with AI Platform. Not all regions provide support for all AI Platform services. For the lastest support per region, see [Region support for AI Platform (Unified) services](https://cloud.google.com/ai-platform-unified/docs/general/locations)

In [ ]:
REGION = 'us-central1' #@param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a custom training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform endpoint based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "ucaip-custom-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://andy-1234-221921ucaip-custom-20210126173733/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Import libraries and define constants

#### Import AI Plarform (Unified) SDK

Import the AI Platform (Unified) SDK into your Python environment.

In [ ]:
import os
import sys
import time
import json
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

from google.cloud.aiplatform import gapic as aip

#### AI Platform (Unified) constants

Let's now setup some constants for AI Platform (Unified):

1. AI Platform (Unified) API service endpoint.
2. AI Platform (Unified) location root path for dataset and model resources.

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# uCAIP location root path for your dataset and model resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### Hardware Accelerators

Let's now set the hardware accelerators (e.g., GPU), if any, for training and prediction.

Set the variables `TRAIN_GPU/TRAIN_NGPU` and `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the compute instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each compute instance, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

For GPU, available accelerators include:
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

   
Otherwise specify `(None, None)` for the container image for a CPU.
   

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)
DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Container (Docker) image

Next, we will set the docker container images for training and prediction.

- Set the variable `TF` to the Tensorflow version of the container image. For example, `2-1` would be version 2.1, and `1-15` would be version 1.15. Google Cloud continuously adds prebuilt training and prediction container images, below are some of the prebuilt images available:

 - Tensorflow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
 - Tensorflow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
 - Tensorflow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23`
   - `gcr.io/cloud-aiplatform/prediction/scikit-learn-cpu.0-23`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-gpu.1-4:latest`
   
Google Cloud AI continously adds new training and prediction container images. For the lastest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers) and [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)

In [ ]:
TF = '2-1'
if TF[0] == '2':
    if TRAIN_GPU:
        TRAIN_VERSION = 'tf-gpu.{}'.format(TF)
    else:
        TRAIN_VERSION = 'tf-cpu.{}'.format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = 'tf2-gpu.{}'.format(TF)
    else:
        DEPLOY_VERSION = 'tf2-cpu.{}'.format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = 'tf-gpu.{}'.format(TF)
    else:
        TRAIN_VERSION = 'tf-cpu.{}'.format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = 'tf-gpu.{}'.format(TF)
    else:
        DEPLOY_VERSION = 'tf-cpu.{}'.format(TF)

TRAIN_IMAGE  = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

Training: gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest None None
Deployment: gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest AcceleratorType.NVIDIA_TESLA_K80 1


#### Machine Type

Next, you will set the machine type (compute instance) you will use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to the compute instance you will use for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]
  
*Note: The following is not supported for training*
 
 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs
 
*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*

In [ ]:
MACHINE_TYPE = 'n1-standard'
VCPU = '4'
TRAIN_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Train Compute Instance', TRAIN_COMPUTE)

MACHINE_TYPE = 'n1-standard'
VCPU = '4'
DEPLOY_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Deploy Compute Instance', DEPLOY_COMPUTE)

Train Compute Instance n1-standard-4
Deploy Compute Instance n1-standard-4


# Tutorial

Now you are ready to start creating your own custom model and training for the Boston Housing data.

## Clients

The AI Platform (Unified) SDK works as a client/server model. On your side, the Python script, you will create a client that sends requests and receives responses from the server -- AI Platform.

You will use several clients in this tutorial, so you will set them all up upfront.

- Job Service for managed models.
- Model Service for managed models.

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(
        client_options=client_options
    )
    return client


def create_model_client():
    client = aip.ModelServiceClient(
        client_options=client_options
    )
    return client


clients = {}
clients['job'] = create_job_client()
clients['model'] = create_model_client()

for client in clients.items():
    print(client)

('job', <google.cloud.aiplatform_v1beta1.services.job_service.client.JobServiceClient object at 0x7f8b90ab51d0>)
('model', <google.cloud.aiplatform_v1beta1.services.model_service.client.ModelServiceClient object at 0x7f8b90ab5190>)


## Prepare your custom job specification

Now that your clients are ready, your first step is to create a Job Specification for your custom training job.

You are going to start with training what we can call an **empty job**. That is, you will create a job specification that provisions resources for training a job, and initiate the job using the client job service -- but the job itself will be empty. 

You do this so you can first focus on understanding the basic steps. Afterwards, you repeat again with a focus on adding the Python training package for training a Boston Housing custom model.

### Define the container specification

Let's first start by defining a job name and then a container specification:

- `JOB_NAME`: A unique name for your custom training job. For convenience, we appended the name with the current datetime to make the name unique.
- `MODEL_DIR`: A location in your Cloud Storage bucket for storing the model artificats.
- `image_uri`: The location of the container image in your local Cloud Storage bucket. This can be either a Google Cloud prebuilt image or your own custom container.
- `--model-dir`: A command line parameter to the container indicating the location to store the model.

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = 'gs://{}/{}'.format(BUCKET_NAME, JOB_NAME)
CONTAINER_SPEC = {
    "image_uri": TRAIN_IMAGE, 
    "args": [
        "--model-dir=" + MODEL_DIR
    ],
}

### Define the worker pool specification


Next, define the worker pool specification for your custom training job. This tells AI Platform what type and how many instances of machines to provision for the training.

For this tutorial, you will use a single instance (node). 

- `replica_count`: The number of instances to provision of this machine type.
- `machine_type`: The type of GCP instance to provision -- e.g., n1-standard-8.
- `accelerator_type`: The type, if any, of hardware accelerator.  In this tutorial if you previously set the variable `TRAIN_GPU != None`, you are using a GPU; otherwise you will use a CPU. 
- `accelerator_count`: The number of accelerators.
- `container_spec`: The docker container to install on the instance(s).

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU
    }
else:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_count": 0
    }

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": CONTAINER_SPEC,
    }
]

If you were doing distributed training, you would add a second machine description and set the replica count accordingly. In the example below, the first machine descrption is the primary (coordinator), and the second ones are the machines the training is distributed to.

```
WORKER_POOL_SPEC=[
     {
        "replica_count": 1,
        "machine_spec": {
          "machine_type": "n1-standard-8"
        },
        "container_spec":  CONTAINER_SPEC,
      },
      {
        "replica_count": 6,
        "machine_spec": {
          "machine_type": "n1-standard-8"
        },
        "container_spec": CONTAINER_SPEC
      }
]
```

### Assemble the job specification

Let's now assemble the description for the custom job specification.

- `display_name`: The human readable name you assign to this custom job.
- `job_spec`: The specification for the custom job. Since this is an empty job, you only specified the resource requirements.

In [ ]:
CUSTOM_JOB = {
    "display_name": JOB_NAME,
    "job_spec": {
        "worker_pool_specs": WORKER_POOL_SPEC
    }
}

## Train the model

Let's now start the training of your custom training job on AI Platform. Use this helper function `create_custom_job`, which takes the parameter:

-`custom_job`: The specification for the custom job.

The helper function uses the job client service and calls the method `create_custom_job`, with the parameters:

-`parent`: The AI Platform (Unified) location path to dataset, model and endpoint resources.
-`custom_job`: The specification for the custom job.

We display a handful of the fields returned in `response` object, with the two that are of most interest are:

`response.name`: The AI Platform (Unified) fully qualified identifier assigned to this custom job. This identifier is saved for using in subsequent steps.

`response.state`: The current state of the custom job. 

In [ ]:
def create_custom_job(custom_job):
    response = clients['job'].create_custom_job(parent=PARENT, custom_job=CUSTOM_JOB)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response.name


# Save the job name
JOB_ID = create_custom_job(CUSTOM_JOB)

name: projects/759209241365/locations/us-central1/customJobs/5510488395622645760
display_name: custom_job_20210126173733
state: JobState.JOB_STATE_PENDING
create_time: 2021-01-26 17:37:36.885697+00:00
update_time: 2021-01-26 17:37:36.885697+00:00


### List all custom jobs

Now that your custom job is running, let's get a list for all your custom jobs -- specific to your `PROJECT_ID`. This will probably be just one job, unless you've been running this tutorial multiple times or otherwise been using the AI Platform (Unified) job service.

Use the helper function `list_custom_jobs`, which uses the job client service and calls the method `list_custom_jobs`. The response object is a list, where each element in the list is a separate job.  

The `response` object for each custom job contains:

- `name`: The AI Platform (Unified) fully qualified identifier for your custom training job.
- `display_name`: The human readable name you assigned to your custom training job.
- `job_spec`: The job specification you provided for your custom training job.
- `state`: The current status of the custom job:
- `start_time`: When the custom training job was created.
- `end_time`: When the execution of the custom job ended.
- `update_time`: When the last time there was a status update to the custom job.

In [ ]:
def list_custom_jobs():
    response = clients['job'].list_custom_jobs(parent=PARENT)
    for job in response:
        print(response)


list_custom_jobs()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Get information on a custom Job

Next, use this helper function `get_custom_job`, which takes the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for the custom job.

The helper function uses the job client service to get the job information for just this job by calling the method `get_custom_job`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for the custom job.

If you recall, you got the AI Platform (Unified) fully qualified identifier for the custom job in the `response.name` field when you called the `create_custom_job` method, and saved the identifier in the variable `JOB_ID`.

In [ ]:
def get_custom_job(name, silent=False):
    response = clients['job'].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


get_custom_job(JOB_ID)

name: projects/759209241365/locations/us-central1/customJobs/5510488395622645760
display_name: custom_job_20210126173733
state: JobState.JOB_STATE_PENDING
create_time: 2021-01-26 17:37:36.885697+00:00
update_time: 2021-01-26 17:37:38.668779+00:00


name: "projects/759209241365/locations/us-central1/customJobs/5510488395622645760"
display_name: "custom_job_20210126173733"
job_spec {
  worker_pool_specs {
    machine_spec {
      machine_type: "n1-standard-4"
    }
    replica_count: 1
    disk_spec {
      boot_disk_type: "pd-ssd"
      boot_disk_size_gb: 100
    }
    container_spec {
      image_uri: "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest"
      args: "--model-dir=gs://andy-1234-221921ucaip-custom-20210126173733/custom_job_20210126173733"
    }
  }
}
state: JOB_STATE_PENDING
create_time {
  seconds: 1611682656
  nanos: 885697000
}
start_time {
  seconds: 1611682657
  nanos: 326204000
}
update_time {
  seconds: 1611682658
  nanos: 668779000
}

### View logs

In addition to the aforementioned state information on the job information, you could also look at logs associated with the custom job.

#### View logs on AI Platform Console

Let's first look at logs generated for AI Platform Console. The cell below will display a link. Paste the link in the address bar of another tab in your browser. It will display information about your job.

In [ ]:
print('https://console.cloud.google.com/ai/platform/locations/{region}/training/{job_id}?project={project_id}'.format(
    region=REGION, job_id=JOB_ID.split('/')[-1],
    project_id=PROJECT_ID))

https://console.cloud.google.com/ai/platform/locations/us-central1/training/5510488395622645760?project=andy-1234-221921


#### View logs on Cloud Logging

Next, let's look at logs generated for Cloud Logging. The cell below will display a link. Paste the link in the address bar of another tab in your browser. It will display logs for your job.

In [ ]:
print('https://console.cloud.google.com/logs/viewer?resource=ml_job%2Fjob_id%2F{job_id}&project={project_id}'.format(
    job_id=JOB_ID.split('/')[-1], project_id=PROJECT_ID))

https://console.cloud.google.com/logs/viewer?resource=ml_job%2Fjob_id%2F5510488395622645760&project=andy-1234-221921


### Cancel a custom job

Next, we will show you how to cancel a custom training job. You will go ahead and cancel your "empty" training job. Use this helper function `cancel_job`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for your custom training job.

The helper function will use the job service client and call the method `cancel_custom_job`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for your custom training job.

We put a try/except around the call since it will throw an exception if the job is already completed (succeeded) -- which most likely it is.

In [ ]:
def cancel_job(name):
    try:
        response = clients['job'].cancel_custom_job(name=name)
        print(response)
    except Exception as e:
        print(e)


time.sleep(10)
cancel_job(JOB_ID)

None


### Delete a custom job

Next, we will show you how to delete a custom training job. You will go ahead and delete your "empty" training job. Use the helper function `delete_job`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for your custom training job.

The helper function will use the job service client and call the method `delete_custom_job`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for your custom training job.

Afterwards, we will verify that the job has been deleted by calling the method `get_custom_job` for the same job. We put a try/except around the call since it will throw an exception if the job is already deleted -- which most likely it is.

In [ ]:
def delete_job(name):
    try:
        response = clients['job'].delete_custom_job(name=name)
        print("Delete", response)
    except Exception as e:
        print(e)

    try:
        response = clients['job'].get_custom_job(name=name)
    except Exception as e:
        print(e)


time.sleep(10)
delete_job(JOB_ID)

400 The CustomJob "projects/759209241365/locations/us-central1/customJobs/5510488395622645760" is in state "PENDING", and cannot be deleted. Please cancel it or wait for its completion before trying deleting it again.


## Train a model - Boston Housing

Now that you have seen the basic steps for custom training, you will do a new custom job to train a model. There are two ways you can train a custom model using a container image:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. This Python package contains your code for training a custom model.

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.

In this tutorial, you will train a Boston Housing model using a Google Cloud prebuilt container.You will need to update the worker pool specification by adding a description for `python_package_spec`. This section will tell the custom job the Python training package to install and which Python module to invoke, along with command line arguments for the Python module.

Let's dive deeper now into the Python package specification:

-`executor_image_spec`: This is the docker image which is configured for your custom training job. You will continue to use the same one you used earlier for demonstration.

-`package_uris`: This is a list of the locations (URIs) of your Python training packages to install on the provisioned instance. The locations need to be in a Cloud Storage bucket. These can be either individual Python files or a zip (archive) of an entire package. In the later case, the job service will unzip (unarchive) the contents into the docker image.

-`python_module`: The Python module (script) to invoke for running the custom training job. In this example, you will be invoking `trainer.task.py` -- note that it was not neccessary to append the `.py` suffix.

-`args`: The command line arguments to pass to the corresponding pythom module. In this example, you will be:
  - `"--model-dir=" + MODEL_DIR` : The Cloud Storage location where to store the model artifacts. There are two ways to tell the training script where to save the model artifacts:
      - direct: You pass the Cloud Storage location as a command line argument to your training script (set variable `DIRECT = True`), or
      - indirect: The service passes the Cloud Storage location as the environment variable `AIP_MODEL_DIR` to your training script (set variable `DIRECT = False`). In this case, you tell the service the model artifact location in the job specification.
  - `"--epochs=" + EPOCHS`: The number of epochs for training.
  - `"--distribute=" + TRAIN_STRATEGY"` : The training distribution strategy to use for single or distributed training.
     - `"single"`: single device.
     - `"mirror"`: all GPU devices on a single compute instance.
     - `"multi"`: all GPU devices on all compute instances.

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU
    }
else:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_count": 0
    }
    
if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"
    
EPOCHS = 20

DIRECT = True
if DIRECT:
    CMDARGS = [
                "--model-dir=" + MODEL_DIR,
                "--epochs=" + str(EPOCHS),
                "--distribute=" + TRAIN_STRATEGY
              ]
else:
    CMDARGS = [
                "--epochs=" + str(EPOCHS),
                "--distribute=" + TRAIN_STRATEGY
              ]


WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": ["gs://" + BUCKET_NAME + "/boston_housing.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        }
    }
]

### Assemble the job specification

Let's now assemble the description for the custom job specification:

- `display_name`: The human readable name you assign to this custom job.
- `job_spec`: The specification for the custom job. 
    - `base_output_directory`: This tells the service the Cloud Storage location where to save the model artifacts (when variable `DIRECT = False`). The service will then pass the location to the training script as the environment variable `AIP_MODEL_DIR`, and the path will be of the form:
    
                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    JOB_SPEC = {
        "worker_pool_specs": WORKER_POOL_SPEC
    }
else:
    JOB_SPEC = {
        "worker_pool_specs": WORKER_POOL_SPEC,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR}
    }
        
CUSTOM_JOB = {
    "display_name": JOB_NAME,
    "job_spec": JOB_SPEC
}

### Examine the training package

#### Package layout

Before you start the training, let's look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the docker image.

The file `trainer/task.py` is the Python script for executing the custom job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Package Assembly

In the following cells, you will assemble the training package.

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
# Requires TensorFlow Datasets\n\
setuptools.setup(\n\
    install_requires=[\n\
        'tensorflow_datasets==1.3.0',\n\
    ],\n\
    packages=setuptools.find_packages())" 
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Custom Training Boston Housing\n\
Version: 0.0.0\n\
Summary: Demostration training script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@google.com\n\
License: Public\n\
Description: Demo\n\
Platform: AI Platform (Unified)"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py contents

In the next cell, you will write the contents of the training script task.py. I won't go into detail, it's just there for you to browse. In summary:

- Get the directory where to save the model artifacts from the command line (`--model_dir`), and if not specified, then from the environment variable `AIP_MODEL_DIR`.
- Loads Boston Housing Prices dataset from tf.keras builtin datasets
- Builds a simple deep neural network model using TF.Keras model API.
- Compiles the model (`compile()`).
- Sets a training distribution strategy according to the argument `args.distribute`.
- Trains the model (`fit()`) with epochs specified by `args.epochs`.
- Saves the trained model (`save(args.model_dir)`) to the specified model directory.

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10
  
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

def make_dataset():
  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  for _ in range(13):
    x_train[_] = scale(x_train[_])
    x_test[_] = scale(x_test[_])
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)


Writing custom/trainer/task.py


#### Store training script on your Cloud Storage bucket

Next, we package the training folder into a compressed tar ball, and then store it in your Cloud Storage bucket.

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/boston_housing.tar.gz

custom/
custom/PKG-INFO
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
custom/setup.cfg
custom/README.md
custom/setup.py
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.8 KiB/  1.8 KiB]                                                
Operation completed over 1 objects/1.8 KiB.                                      


### Train the model

Let's now start the training of your custom training job on AI Platform. Use the helper function defined earlier `create_custom_job`, which will return the AI Platform (Unified) fully qualified identifier assigned to the custom job.

In [ ]:
# Save the job name
JOB_ID = create_custom_job(CUSTOM_JOB)

name: projects/759209241365/locations/us-central1/customJobs/159086138399653888
display_name: custom_job_20210126173733
state: JobState.JOB_STATE_PENDING
create_time: 2021-01-26 17:38:03.576114+00:00
update_time: 2021-01-26 17:38:03.576114+00:00


### Get information on the custom job

Let's get the status on your custom training job for CIFAR10 using the helper function we defined earlier `get_custom_job`. The job most likely will still be either PENDING or RUNNING.


In [ ]:
result = get_custom_job(JOB_ID)

name: projects/759209241365/locations/us-central1/customJobs/159086138399653888
display_name: custom_job_20210126173733
state: JobState.JOB_STATE_RUNNING
create_time: 2021-01-26 17:38:03.576114+00:00
update_time: 2021-01-26 17:38:03.806481+00:00


# Deployment

## Pre-Cooked

Training the above model may take upwards of ~5 minutes time. For expendiency, we have a pre-cooked (already trained) version of this model you can use for the next steps, while you wait for your model to finish training. 

Once your model is done training, you can repeat these steps for your trained model. You can calcuate the actual time it took to train the model by subtracting `end_time` from `start_time`. For your model, you will need to know the location of the saved model, which the Python script saved in your local Cloud Storage bucket at `MODEL_DIR + '/saved_model.pb'`.


You can choose between the precooked model or your trained model with the Python variable `precooked` in the cell below.

In [ ]:
# Precooked flag
precook = False

if precook:
    model_path_to_deploy = "[not-implemented-yet]"
else:
    while True:
        response = get_custom_job(JOB_ID, True)
        if response.state != aip.JobState.JOB_STATE_SUCCEEDED: 
            print("Training job has not completed:", response.state)
            model_path_to_deploy = None
            if response.state == aip.JobState.JOB_STATE_FAILED:
                break
        else:
            if not DIRECT:
                MODEL_DIR = MODEL_DIR + "/model"
            model_path_to_deploy = MODEL_DIR
            print("Training Time:", response.update_time - response.create_time)
            break
        time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

Training job has not completed: JobState.JOB_STATE_RUNNING
Training job has not completed: JobState.JOB_STATE_PENDING
Training job has not completed: JobState.JOB_STATE_PENDING
Training job has not completed: JobState.JOB_STATE_PENDING
Training job has not completed: JobState.JOB_STATE_PENDING
Training job has not completed: JobState.JOB_STATE_PENDING
Training job has not completed: JobState.JOB_STATE_PENDING
Training job has not completed: JobState.JOB_STATE_PENDING
Training Time: 0:07:05.592888
model_to_deploy: gs://andy-1234-221921ucaip-custom-20210126173733/custom_job_20210126173733


## Load the saved model

Your model is stored in a TF SavedModel format in a Cloud Storage bucket. Let's go ahead and load it from the Cloud Storage bucket, and then you can do some things, like evaluate the model, and do a prediction.

To load, you will use the TF.Keras `model.load_model()` method passing it the Cloud Storage path where the model is saved -- specified by `MODEL_DIR`.

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f8b0029d250> and <tensorflow.python.keras.layers.core.Dense object at 0x7f8b00259950>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f8b00259950> and <tensorflow.python.keras.layers.core.Dense object at 0x7f8b002669d0>).


## Evaluate the model

Now let's find out how good the model is. 

### Load evaluation data

You will load the Boston Housing test (holdout) data from `tf.keras.datasets`, using the method `load_data()`. This will return the dataset as a tuple of two elements. The first element is the training data and the second is the test data. Each element is also a tuple of two elements: the feature data, and the corresponding labels (median value of owner-occupied home).

You don't need the training data, and hence why we loaded it as `(_, _)`.

Before you can run the data through evaluation, you need to preprocess it:

x_test:
1. Normalize (rescaling) the data in each column by dividing each value by the maximum value of that column. This will replace each single value with a 32-bit floating point number between 0 and 1.

In [ ]:
from tensorflow.keras.datasets import boston_housing
import numpy as np

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113)

def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)

(102, 13) float32 (102,)


### Evaluate the model

Let's evaluate how well the DNN model in the custom job did. 

In [ ]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 4ms/step - loss: 104.0941


104.0941162109375

## Upload the model for serving


Next, you will upload your TF.Keras model from the custom job to AI Platform (Unified) model service, which will create a AI Platform (Unified) model resource for your custom model. You will also need to know the name (signature) of the model's serving input layer. You will use this subsequently when making a prediction request.


## Get the serving function signature

You can get the signatures of your model's input and output layers by reloading the model into memory, and querying it for the signatures corresponding to each layer.

When making a prediction request, you need to route the request to the serving function instead of the model, so you need to know the input layer name of the serving function -- which you will use later when you make a prediction request.

You will also need to know the name of the serving function's input and output layer for constructing the explanation metadata -- which is discussed subsequently.

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]
print('Serving function input:', serving_input)
serving_output = list(loaded.signatures['serving_default'].structured_outputs.keys())[0]
print('Serving function output:', serving_output)

input_name = model.input.name
print('Model input name:', input_name)
output_name = model.output.name
print('Model output name:', output_name)

Serving function input: dense_input
Serving function output: dense_2
Model input name: input_1:0
Model output name: dense_2/Identity:0


### Explanation Specification

To get explanations when doing a prediction, you must enable the explanation capability and set corresponding settings when you upload your custom model to a AI Platform (Unified) managed model. These settings are referred to as the explanation metadata, which consists of:

- `parameters`: This is the specification for the explainability algorithm to use for explanations on your model. You can choose between:
  - Shapley - *Note*, not recommended for image data -- can be very long running
  - XRAI
  - Integrated Gradients
- `metadata`: This is the specification for how the algoithm is applied on your custom model.

#### Explanation Parameters

Let's first dive deeper into the settings for the explainability algorithm.

#### Shapley

Assigns credit for the outcome to each feature, and considers different permutations of the features. This method provides a sampling approximation of exact Shapley values.

Use Cases: 
  - Classification and regression on tabular data.

Parameters:

- `path_count`: This is the number of paths over the features that will be processed by the algorithm. An exact approximation of the Shapley values requires M! paths, where M is the number of features. For the Boston Housing dataset, this would be 13.

For any non-trival number of features, this is too compute expensive. You can reduce the number of paths over the features to M * `path_count`.

#### Integrated Gradients

A gradients-based method to efficiently compute feature attributions with the same axiomatic properties as the Shapley value.

Use Cases: 
  - Classification and regression on tabular data. 
  - Classification on image data.

Parameters:

- `step_count`: This is the number of steps to approximate the remaining sum. The more steps, the more accurate the integral approximation. The general rule of thumb is 50 steps, but as you increase so does the compute time.

#### XRAI

Based on the integrated gradients method, XRAI assesses overlapping regions of the image to create a saliency map, which highlights relevant regions of the image rather than pixels.

Use Cases: 

  - Classification on image data.

Parameters:

- `step_count`: This is the number of steps to approximate the remaining sum. The more steps, the more accurate the integral approximation. The general rule of thumb is 50 steps, but as you increase so does the compute time.

In the next code cell, set the variable `XAI` to which explainabilty algorithm you will use on your custom model.

In [ ]:
XAI = "shapley"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {
        "sampled_shapley_attribution": { 'path_count': 10 }
    }
elif XAI == "ig":
    PARAMETERS = {
        "integratedGradientsAttribution": { "step_count": 50 }
    }
elif XAI == "xrai":
    PARAMETERS = {
        "xrai": { "step_count": 50 }
    }
    
parameters = aip.ExplanationParameters(PARAMETERS)

#### Explanation Metadata

Let's first dive deeper into the explanation metadata, which consists of:

- `outputs`: A scalar value in the output to attribute -- what to explain. For example, in a probability output \[0.1, 0.2, 0.7\] for classification, one wants an explanation for 0.7. Consider the following formulae, where the output is `y` and that is what we want to explain.

    y = f(x)
    
Consider the following formulae, where the outputs are `y` and `z`. Since we can only do attribution for one scalar value, we have to pick whether we want to explain the output `y` or `z`. Assume in this example the model is object detection and y and z are the bounding box and the object classification. You would want to pick which of the two outputs to explain.

    y, z = f(x)
    
The dictionary format for `outputs` is:

    { "outputs": { "<your_display_name>": 
                        "output_tensor_name": <layer>
                 }
    }
  
<blockquote>
 -  [your_display_name]: A human readable name you assign to the output to explain. A common example is "probability".<br/>
 -  "output_tensor_name": The key/value field to identify the output layer to explain. <br/>
 -  [layer]: The output layer to explain. In a single task model, like a tabular regressor, it is the last (topmost) layer in the model.
</blockquote>
    
- `inputs`: The features for attribution -- how they contributed to the output. Consider the following formulae, where `a` and `b` are the features. We have to pick which features to explain how the contributed. Assume that this model is deployed for A/B testing, where `a` are the data_items for the prediction and `b` identifies whether the model instance is A or B. You would want to pick `a` (or some subset of) for the features, and not `b` since it does not contribute to the prediction.

    y = f(a,b)
    
The minimum dictionary format for `inputs` is:

    { "inputs": { "<your_display_name>": 
                        "input_tensor_name": <layer>
                 }
    }
  
<blockquote>
 -  [your_display_name]: A human readable name you assign to the input to explain. A common example is "features".<br/>
 -  "input_tensor_name": The key/value field to identify the input layer for the feature attribution. <br/>
 -  [layer]: The input layer for feature attribution. In a single input tensor model, it is the first (bottom-most) layer in the model.
</blockquote>

Since the inputs to the model are tabular, you can specify the following two additional fields as reporting/visualization aids:

<blockquote>
 - "encoding": "BAG_OF_FEATURES" : Indicates that the inputs are set of tabular features.<br/>
 - "index_feature_mapping": [ feature-names ] : A list of human readable names for each feature. For this example, we use the feature names specified in the dataset.<br/>
 - "modality": "numeric": Indicates the field values are numeric.
</blockquote>

In [ ]:
INPUT_METADATA = {'input_tensor_name': serving_input,
                  "encoding": "BAG_OF_FEATURES",
                  "modality": "numeric",
                  "index_feature_mapping": ["crim", "zn", "indus", "chas", "nox", "rm", "age", 
                                            "dis", "rad", "tax", "ptratio", "b", "lstat"]}

OUTPUT_METADATA = {'output_tensor_name': serving_output}

input_metadata = aip.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.ExplanationMetadata(
        inputs={'features': input_metadata},
        outputs={'medv' : output_metadata}
)

explanation_spec = aip.ExplanationSpec(
    metadata=metadata,
    parameters=parameters
)

## Deploy the model for batch prediction

Let's now deploy the trained AI Platform (Unified) model you created with Custom Job for batch prediction. This differs from deploying a model for on-demand prediction.

For on-demand prediction, you:

1. Create an endpoint for deploying the model to.

2. Deploy the model to the endpoint.

3. Make on-demand (live) prediction requests to the endpoint.

For batch-prediction, you:

1. Create a batch prediction job.

2. The job service will provision resources for the batch prediction request.

3. The results of the batch prediction request are returned to the caller.

4. The job service will unprovision the resoures for the batch prediction request.

### Upload the model

First, use this helper function `upload_model` to upload your model, stored in SavedModel format, up to the model service, which will instantiate a AI Platform (Unified) model instance for our model. Once you've done that, you can use the model in the same way as any other AI Platform (Unified) model instance, such as deploying to an endpoint for serving predictions.

The helper function takes the parameters:

- `display_name`: A human readable name for the endpoint.
- `image_uri`: The container image for the model deployment.
- `model_uri`: The Cloud Storage path to our SavedModel artificat. For this tutorial, this is the Cloud Storage location where the `trainer/task.py` saved the model, which you specified in the variable `MODEL_DIR`.

The helper function uses the model client service and calls the method `upload_model`, which takes the parameters:

- `parent`: The AI Platform (Unified) location root path for dataset, model and endpoint resources. 
- `model`: The specification for the AI Platform (Unified) model instance.

Let's now dive deeper into the AI Platform (Unified) model specification `model`. This is a dictionary object that consists of the following fields:

- `display_name`: A human readable name for the model.
- `metadata_schema_uri`: Since our model was built without a AI Platform (Unified) managed dataset, you will leave this blank (`''`).
- `artificat_uri`: The Cloud Storage path where the model is stored in SavedModel format. 
- `container_spec`: This is the specification for the docker container that will be installed on the endpoint, from which the model will serve predictions. Use the variable you set earlier `DEPLOY_GPU != None` to use a GPU; otherwise only a CPU is allocated.
- `explanation_spec`: This is the specification for enabling explainability for your model.

Uploading a model into a AI Platform (Unified) model resource returns a long running operation, since it may take a few moments. You call `response.result()`, which is a synchronous call and will return when the AI Platform (Unified) model resource is ready. 

The helper function returns the AI Platform (Unified) fully qualified identifier for the corresponding AI Platform (Unified) model instance `upload_model_response.model`. You will save the identifier for subsequent steps in the variable `model_to_upload_name`.


In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    
    model = aip.Model(display_name=display_name,
                      artifact_uri=model_uri,
                      metadata_schema_uri="",
                      explanation_spec=explanation_spec,
                      container_spec={
                        "image_uri": image_uri
                     })

    response = clients['model'].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_name = upload_model("boston-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy)

Long running operation: projects/759209241365/locations/us-central1/models/2003503699854360576/operations/2036372500455096320
upload_model_response
 model: projects/759209241365/locations/us-central1/models/2003503699854360576


### List all models

Now that your custom model is uploaded as a AI Platform (Unified) managed model, let's get a list of all your AI Platform (Unified) managed models. Use this helper function `list_models`. This helper function uses the AI Platform (Unified) model client service, and calls the method `list_models`, with the parameter:

- `parent`: The AI Platform (Unified) location root path for your dataset, model and endpoint resources.

The response object from the call is a list, where each element is a AI Platform (Unified) managed model. For each model, you will display a few fields:

- `name`: The AI Platform (Unified) unique identifier for the managed model.
- `display_name`: The human readable name assigned to the model.
- `create_time`': Timestamp when the model resource was created.
- `update_time`: Timestamp when the model resource was last updated.
- `container`: The container image used for training the model.
- `artifact_uri`': The Cloud Storage location of the model artifact.

In [ ]:
def list_models():
    response = clients['model'].list_models(parent=PARENT)
    for model in response:
        print("name", model.name)
        print("display_name", model.display_name)
        print("create_time", model.create_time)
        print("update_time", model.update_time)
        print("container", model.container_spec.image_uri)
        print("artifact_uri", model.artifact_uri)
        print('\n')


list_models()

name projects/759209241365/locations/us-central1/models/2003503699854360576
display_name boston-20210126173733
create_time 2021-01-26 17:47:03.240420+00:00
update_time 2021-01-26 17:47:20.022104+00:00
container gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest
artifact_uri gs://andy-1234-221921ucaip-custom-20210126173733/custom_job_20210126173733


name projects/759209241365/locations/us-central1/models/4579562686710284288
display_name cifar10-20210125193549
create_time 2021-01-25 19:47:46.729024+00:00
update_time 2021-01-25 19:48:06.101088+00:00
container gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest
artifact_uri gs://andy-1234-221921ucaip-custom-20210125193549/custom_job_20210125193549


name projects/759209241365/locations/us-central1/models/8038327200530825216
display_name iris_model-20210125190508
create_time 2021-01-25 19:05:47.497398+00:00
update_time 2021-01-25 19:36:36.026658+00:00
container 
artifact_uri 


name projects/759209241365/locations/us-central1/models/7

### Get model information

Now let's get the model information for just your model. Use this helper function `get_model`, with the parameter:

- `name`: The AI Platform (Unified) unique identifier for the managed model.

This helper function uses the AI Platform (Unified) model client service, and calls the method `get_model`, with the parameter:

- `name`: The AI Platform (Unified) unique identifier for the managed model.

In [ ]:
def get_model(name):
    response = clients['model'].get_model(name=name)
    print(response)


get_model(model_to_deploy_name)

name: "projects/759209241365/locations/us-central1/models/2003503699854360576"
display_name: "boston-20210126173733"
predict_schemata {
}
metadata {
}
container_spec {
  image_uri: "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
create_time {
  seconds: 1611683223
  nanos: 240420000
}
update_time {
  seconds: 1611683240
  nanos: 22104000
}
etag: "AMEw9yPwBFLZJu9tpbMWaarYFfv20qmz0RIQljVarDtMrPgntTK1bWgy5Qb48frpVjk="
supported_export_formats {
  id: "custom-trained"
  exportable_contents: ARTIFACT
}
explanation_spec {
  parameters {
    sampled_shapley_attribution {
      path_count: 10
    }
  }
  metadata {
    inputs {
      key: "features"
      value {
      

### Get test item(s)

Let's now do a prediction to your deployed model. You will use an arbitrary data items out of the test (holdout) portion of the dataset as the test data. 

In [ ]:
test_item_1 = x_test[0]
test_label_1 = y_test[0]
test_item_2 = x_test[1]
test_label_2 = y_test[1]

print(test_item_1.shape)

(13,)


You then write the prediction request as a JSONL file to a Cloud Storage bucket. Each instance in the prediction request is a dictionary entry of the form:

                        {input_name: content}
                        
- `input_name`: the name of the input layer of the underlying model.
- `content`: The feature values of the test item as a list.

In [ ]:
gcs_input_uri = "gs://" + BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    data = {serving_input: test_item_1.tolist()}
    f.write(json.dumps(data) + '\n')

### Compute instance scaling

You have several choices on scaling the compute instances for handling your batch request:

- Single Instance: The batch request is processed on a single compute instance.
  - Set the minimum (`MIN_NODES`) and maximum (`MAX_NODES`) number of compute instances to one. 


- Manual Scaling: The batch request is split across a fixed number of compute instances that you manually specified.
  - Set the minimum (`MIN_NODES`) and maximum (`MAX_NODES`) number of compute instances to the same number of nodes. When the batch is processed, the fixed number of compute instances are provisioned and the batch request is evenly distributed across them.
  
  
- Auto Scaling: The batch request is streamed across an initial number of compute instances and then based on time and compute tradeoff may be automatically scaled to additionally compute instances.
  - Set the minimum (`MIN_NODES`) to the initial number of compute instances and the maximum (`MAX_NODES`) to the maximum number of nodes the service could auto-scale to.

The minimum number of compute instances corresponds to the field `starting_replica_count` and the maximum number of compute instances corresponds to the field `max_replica_count`, in your subsequent batch request.

When the batch request is completed, the compute instances are de-provisioned.

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### Make batch prediction with explainability request

Now that your batch of two image test items is ready, let's do the batch request. Use this helper function `create_batch_prediction_job`, with the parameters:

- `display_name`: The human readable name for the prediction job.
- `model_name`: The AI Platform (Unified) fully qualified identifier for the model.
- `gcs_source_uri`: The Cloud Storage path to the JSONL/CSV input file -- which you created above.
- `gcs_destination_output_uri_prefix`: The Cloud Storage path that the service will write the predictions to.

The helper function uses the job client service and calls the method `create_batch_prediction_job`, with the parameters:

- `parent`: The AI Platform (Unified) location root path for dataset, model and pipeline resources.
- `batch_prediction_job`: The specification for the batch prediction job.

Let's now dive into the specification for the `batch_prediction_job`:

- `display_name`: The human readable name for the prediction batch job.
- `model`: The AI Platform (Unified) fully qualified identifier for the model.
- `model_parameters`: requirements/constrains on the prediction service.
 - `confidenceThreshold`: The minimum confidence threshold on doing a prediction.
 - `maxPredictions`: The maximum size of the batch request.
- `input_config`: The input source and format type for the instances to predict.
 - `instances_format`: The format of the batch prediction request file: `csv` or `jsonl`.
 - `gcs_source`: A list of one or more Cloud Storage paths to your batch prediction requests.
- `output_config`: The output destination and format for the predictions.
 - `prediction_format`: The format of the batch prediction response file: `csv` or `jsonl`.
 - `gcs_destination`: The output destination for the predictions.
- `dedicated_resources`: The compute resources to provision for the batch prediction job. 
  - `machine_spec`: The compute instance to provision. Use the variable `DEPLOY_GPU != None` to use a GPU; otherwise only a CPU is allocated.
  - `starting_replica_count`: The number of compute instances to initially provision.
  - `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.
- `generate_explanation`: Return explanations with the predictions.

This call is an asychronous operation. You will print from the response object a few select fields, including:

- `name`: The AI Platform (Unified) fully qualified identifier assigned to the batch prediction job.
- `display_name`: The human readable name for the prediction batch job.
- `model`: The AI Platform (Unified) fully qualified identifier for the model.
- `generate_explanations`: Whether True/False explanations were provided with the predictions (explainability).
- `state`: The state of the prediction job (pending, running, etc).

Since this call will take a few moments to execute, you will likely get `JobState.JOB_STATE_PENDING` for `state`.

The helper function will return and save the AI Platform (Unified) fully qualified identifier assigned to the batch prediction job as `prediction_name`.

In [ ]:
BATCH_MODEL = "boston_batch-" + TIMESTAMP


def create_batch_prediction_job(display_name, model_name, gcs_source_uri, gcs_destination_output_uri_prefix):

    model_parameters = {
        "confidenceThreshold": 0.5,
        "maxPredictions": 10000,
    }

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(model_parameters, Value()),
        "input_config": {
            "instances_format": "jsonl",
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": "jsonl",
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
        "generate_explanation": True
    }
    response = clients['job'].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    print(" generate_explanation:", response.generate_explanation)
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


response = create_batch_prediction_job(BATCH_MODEL, model_to_deploy_name, gcs_input_uri, "gs://" + BUCKET_NAME)

prediction_name = response.name

response
 name: projects/759209241365/locations/us-central1/batchPredictionJobs/3577881205527281664
 display_name: boston_batch-20210126173733
 model: projects/759209241365/locations/us-central1/models/2003503699854360576
 generate_explanation: True
 state: JobState.JOB_STATE_PENDING
 create_time: 2021-01-26 22:22:54.982819+00:00
 start_time: None
 end_time: None
 update_time: 2021-01-26 22:22:54.982819+00:00
 labels: {}


### List all batch prediction jobs

Use this helper function `list_batch_prediction_jobs`. This helper function uses the job client service and calls the method `list_batch_prediction_jobs`, with the parameter:

- `parent`: The AI Platform (Unified) location root path to the dataset, model and pipeline resources.

The method will return a list, where each element is a single batch prediction job. You will probably only have one, unless you've already been using the service or been experimenting with this tutorial.

We will print a couple of additional fields:

- `error`: An error description if an error occurred.
- `output_uri_prefix`: The Cloud Storage location you gave for outputtng the predictions.

In [ ]:
def list_batch_prediction_jobs():
    response = clients['job'].list_batch_prediction_jobs(parent=PARENT)
    for batch in response:
        print(" name:", batch.name)
        print(" display_name:", batch.display_name)
        print(" model:", batch.model)
        print(" generate_explanation:", batch.generate_explanation)
        print(" state:", batch.state)
        print(" error:", batch.error)
        gcs_destination = batch.output_config.gcs_destination
        print(" gcs_destination")
        print("  output_uri_prefix:", gcs_destination.output_uri_prefix)


list_batch_prediction_jobs()

 name: projects/759209241365/locations/us-central1/batchPredictionJobs/3212526685756850176
 display_name: boston_batch-20210126173733
 model: projects/759209241365/locations/us-central1/models/2003503699854360576
 generate_explanation: True
 state: JobState.JOB_STATE_RUNNING
 error: 
 gcs_destination
  output_uri_prefix: gs://andy-1234-221921ucaip-custom-20210126173733
 name: projects/759209241365/locations/us-central1/batchPredictionJobs/5276301214999379968
 display_name: iris_batch-20210125190508
 model: projects/759209241365/locations/us-central1/models/8038327200530825216
 generate_explanation: True
 state: JobState.JOB_STATE_SUCCEEDED
 error: 
 gcs_destination
  output_uri_prefix: gs://andy-1234-221921ucaip-automl-20210125190508
 name: projects/759209241365/locations/us-central1/batchPredictionJobs/4198146501081300992
 display_name: cifar10_batch-20210112180452
 model: projects/759209241365/locations/us-central1/models/699430132753891328
 generate_explanation: True
 state: JobStat

### Get information on a batch prediction job

Use this helper function `get_batch_prediction_job`, with the paramter:

- `job_name`: The AI Platform (Unified) fully qualified identifier for the batch prediction job.

The helper function uses the job client service and calls the method `get_batch_prediction_job`, with the paramter:

- `name`: The AI Platform (Unified) fully qualified identifier for the batch prediction job. In this tutorial, you will pass it the AI Platform (Unified) fully qualified identifier for your batch prediction job -- `prediction_name`

The helper function will return the Cloud Storage path to where the predictions are stored -- `gcs_destination`.

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients['job'].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    print(" generate_explanation:", response.generate_explanation)
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(prediction_name)

response
 name: projects/759209241365/locations/us-central1/batchPredictionJobs/3577881205527281664
 display_name: boston_batch-20210126173733
 model: projects/759209241365/locations/us-central1/models/2003503699854360576
 generate_explanation: True
 state: JobState.JOB_STATE_RUNNING
 error: 
 gcs_destination
  output_uri_prefix: gs://andy-1234-221921ucaip-custom-20210126173733


### Get the predictions

When the batch prediction is done processing, the job state will be `JOB_STATE_SUCCEEDED`.

Finally you view the predictions stored at the Cloud Storage path you set as output. The predictions will be in a JSONL format, which you indicated at the time you made the batch prediction job, under a subfolder starting with the name `prediction`, and under that folder will be a file called `prediction.results-xxxx-of-xxxx`.

Let's display (cat) the contents. You will a row for each prediction -- in this case, there is just one row. The row contains:

- `dense_input`: The input for the prediction.
- `prediction`: The predicted median housing value in 1K USD unit.

In [ ]:
def get_lastest_predictions(gcs_out_dir):
    ''' Get the latest prediction subfolder using the timestamp in the subfolder name'''
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split('/')[-2]
        if subfolder.startswith('prediction-'):
            if subfolder > latest:
                latest = folder[:-1]
    return latest
    
while True:
    predictions, state = get_batch_prediction_job(prediction_name, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_lastest_predictions(predictions)
        ! gsutil ls $folder/explanation.results*

        ! gsutil cat $folder/explanation.results*
        break
    time.sleep(60)

The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: JobState.JOB_STATE_RUNNING
The job has not completed: J

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- Endpoint
- GCS Bucket

In [ ]:
delete_dataset = True
delete_model = True
delete_batch = True
delete_bucket = True

# Delete the dataset using the uCAIP fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['dataset'].delete_dataset(name=dataset['name'])
except Exception as e:
    print(e)

# Delete the model using the uCAIP fully qualified identifier for the model
try:
    if delete_model:
        clients['model'].delete_model(name=model_to_deploy_name)
except Exception as e:
    print(e)

# Delete the batch prediction job using the AI Platform (Unified) fully qualified identifier for the batch job
try:
    if delete_batch:
        clients['job'].delete_batch_prediction_job(name=prediction_name)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
        
# Collect any unclaimed memory
import gc
gc.collect()